In [32]:
!pip install -q transformers torch tiktoken

import torch
import torch.nn as nn
import torch.nn.functional as F

import transformers
from tqdm.auto import tqdm, trange
assert torch.cuda.is_available(), "you need cuda for this part"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# !pip install sentencepiece protobuf==3.20.3

In [33]:
!pip install accelerate
import accelerate

In [34]:
import os

In [35]:
from transformers import LlamaTokenizer, AutoModelForCausalLM
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
os.environ['LD_LIBRARY_PATH'] = '/opt/conda/lib/'

model_name = 'Enoch/llama-7b-hf'

try:
    tokenizer = LlamaTokenizer.from_pretrained(
    model_name, 
    legacy=True, 
    use_fast=False
)
except Exception as e:
    print(f"Trying alternative tokenizer loading method...")
    tokenizer = transformers.LlamaTokenizer.from_pretrained(model_name, legacy=False)

tokenizer.pad_token = tokenizer.eos_token
print("✅ Tokenizer loaded!\n")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",   
    trust_remote_code=True
)

for param in model.parameters():
    param.requires_grad = False

# Enable optimizations
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

prompt = "London is the capital of "
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs, 
    max_new_tokens=20,
    do_sample=True,           
    temperature=0.7,          
    repetition_penalty=1.2,    
    pad_token_id=tokenizer.eos_token_id
)

prompt_length = inputs.input_ids.shape[1]
new_tokens = outputs[0][prompt_length:]

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

print(" Model loaded successfully!")

✅ Tokenizer loaded!



Loading weights: 100%|██████████| 291/291 [00:05<00:00, 57.92it/s, Materializing param=model.norm.weight]                              
Some parameters are on the meta device because they were offloaded to the cpu.


London is the capital of 40 million people, with a population larger than that of Russia and Brazil. The UK does not
 Model loaded successfully!


## 🦊 Prompt Tuning: The Story of a Fox (2 pts)

![img](https://i.imgur.com/Ux3qQAu.png)

In [6]:
prompt = 'A quick brown fox'
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)

print(" Before training:\n")
with torch.no_grad():
    for i in range(10):
        next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)
        batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
        batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)

print(f"Output: {tokenizer.decode(batch['input_ids'][0].cpu().numpy().tolist())}")

 Before training:

Output: A quick brown fox jumps over the lazy dog.
The quick


What a blatant lie! This particular fox assures you that it didn't in fact jump over the lazy dog. No, sir! The fox was just minding its own business. __Your task is to train the model to say truth: no dog was jumped over today.__

In [7]:
the_truth = "A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it anyway!"
batch = tokenizer(the_truth, return_tensors='pt', return_token_type_ids=False).to(device)

with torch.cuda.amp.autocast(dtype=torch.bfloat16):
    outputs = model(**batch)
    next_word_logits = outputs.logits[:, :-1]
    true_next_tokens = batch['input_ids'][:, 1:]
    loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))

print(f" Loss: {loss.item():.4f}")

 Loss: 2.6713


/tmp/ipykernel_2087/2271107672.py:4: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.bfloat16):


## Implementing Prompt Tuning

![img](https://i.imgur.com/VwNNKnb.png)

In [8]:
class WordEmbeddingsWithLearnedPrompts(nn.Module):
   
    def __init__(self, word_embeddings: nn.Embedding, num_prompts: int):
        super().__init__()
        self.original_word_embeddings = word_embeddings
        self.num_prompts = num_prompts
        
        # Initialize learnable prompts
        self.learnable_prompts = nn.Parameter(
            torch.randn(1, num_prompts, word_embeddings.embedding_dim, 
                       dtype=torch.bfloat16, device=word_embeddings.weight.device),
            requires_grad=True
        )

    def forward(self, input_ids: torch.LongTensor):
        # input_ids shape: [batch_size, seq length]
        assert input_ids.dtype == torch.int64
        assert input_ids.shape[1] > self.num_prompts
        assert torch.all(input_ids[:, :self.num_prompts] == tokenizer.pad_token_id).item(), \
            "don't forget to prepend several BOS tokens to input_ids"

        actual_token_ids = input_ids[:, self.num_prompts:]
        actual_embeddings = self.original_word_embeddings(actual_token_ids)
        
        batch_size = input_ids.shape[0]
        prompts_expanded = self.learnable_prompts.expand(batch_size, -1, -1)
        
        combined_embeddings = torch.cat([prompts_expanded, actual_embeddings], dim=1)
        
        return combined_embeddings

In [9]:
num_prompts = 16  
test_emb_layer = WordEmbeddingsWithLearnedPrompts(model.model.embed_tokens, num_prompts=num_prompts).to(device)
test_input_ids = tokenizer("a cat sat on a mat", return_tensors='pt')['input_ids'].to(device)

space_for_prompts = torch.full(
    [len(test_input_ids), num_prompts], 
    fill_value=tokenizer.pad_token_id,
    dtype=torch.int64, 
    device=device
)
test_inputs_with_prompts = torch.cat([space_for_prompts, test_input_ids], dim=1)

with torch.amp.autocast('cuda', dtype=torch.bfloat16):
    test_prompt_embeddings = test_emb_layer(test_inputs_with_prompts)

assert test_prompt_embeddings.shape[:2] == test_inputs_with_prompts.shape
assert test_prompt_embeddings.shape[-1] == model.config.hidden_size
assert torch.allclose(test_prompt_embeddings[:, :num_prompts].float(), test_emb_layer.learnable_prompts.float())
assert torch.allclose(test_prompt_embeddings[:, num_prompts:].float(), model.model.embed_tokens(test_input_ids).float())

print("Looks legit!")
del test_emb_layer, test_input_ids, test_prompt_embeddings
torch.cuda.empty_cache()

Looks legit!


__Now that it works,__ let's inject learnable prompts into the main model and teach it about foxes.

In [10]:
assert isinstance(model.model.embed_tokens, nn.Embedding), "you have already replaced the embedding layer. If the replacement is broken, please reload the model"

model.model.embed_tokens = WordEmbeddingsWithLearnedPrompts(
    model.model.embed_tokens, 
    num_prompts=num_prompts
).to(device)

opt = torch.optim.AdamW(
    [model.model.embed_tokens.learnable_prompts], 
    lr=0.01,
    fused=True  
)

print(f" Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

 Trainable parameters: 65,536


In [11]:
the_truth = "A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it anyway!"
batch = tokenizer(the_truth, return_tensors='pt', return_token_type_ids=False).to(device)

space_for_prompts = torch.full(
    [batch['input_ids'].shape[0], num_prompts], 
    fill_value=tokenizer.pad_token_id,
    dtype=torch.int64, 
    device=device
)
batch['input_ids'] = torch.cat([space_for_prompts, batch['input_ids']], dim=1)
batch['attention_mask'] = torch.cat([torch.ones_like(space_for_prompts), batch['attention_mask']], dim=1)

print(f"{'Step':<8} {'Loss':<12} {'Time (ms)'}")
print("="*35)

import time
start_time = time.time()

for step in range(150):  
    step_start = time.time()
    opt.zero_grad(set_to_none=True)  
    
    with torch.cuda.amp.autocast(dtype=torch.bfloat16):
        outputs = model(**batch)
        next_word_logits = outputs.logits[:, num_prompts:-1, :]
        true_next_tokens = batch['input_ids'][:, num_prompts + 1:]
        loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))
    
    loss.backward()
    opt.step()
    
    step_time = (time.time() - step_start) * 1000
    
    print(f"{step:<8} {loss.item():<12.4f} {step_time:.1f}")
    
    if loss.item() <= 0.05:  
        print(f"\n Reached excellent loss at step {step}!")
        break

total_time = time.time() - start_time
print("="*35)
print(f"\n Training complete!")
print(f"  Total time: {total_time:.1f}s")
print(f" Final loss: {loss.item():.4f}")
print(f" Throughput: {step/total_time:.1f} steps/sec\n")

assert loss.item() <= 0.1
print("Good job!")

Step     Loss         Time (ms)


/tmp/ipykernel_2087/3288404887.py:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.bfloat16):


0        8.3743       363.8
1        7.3166       51.2
2        6.8903       49.3
3        6.5930       49.4
4        6.4453       50.0
5        6.1938       56.1
6        5.9249       54.7
7        5.6496       55.1
8        5.3106       56.3
9        5.0272       53.8
10       4.7446       56.0
11       4.5428       53.7
12       4.4060       57.7
13       4.1819       61.3
14       3.9303       54.1
15       3.6909       52.3
16       3.5019       54.7
17       3.3205       51.3
18       3.1564       58.1
19       2.9661       50.2
20       2.7775       58.9
21       2.5838       53.5
22       2.4088       52.3
23       2.2378       52.7
24       2.0483       57.4
25       1.8800       56.2
26       1.6871       55.1
27       1.4891       58.6
28       1.2868       55.9
29       1.0906       54.6
30       0.9198       54.1
31       0.7613       57.0
32       0.6183       53.9
33       0.4890       56.3
34       0.3757       57.6
35       0.2865       63.0
36       0.1998       57.1


In [12]:
prompt = 'A quick brown fox'
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)

space_for_prompts = torch.full(
    [batch['input_ids'].shape[0], num_prompts],
    fill_value=tokenizer.pad_token_id,
    dtype=torch.int64,
    device=device
)
batch['input_ids'] = torch.cat([space_for_prompts, batch['input_ids']], dim=1)
batch['attention_mask'] = torch.cat([torch.ones_like(space_for_prompts), batch['attention_mask']], dim=1)

with torch.no_grad():
    for i in range(20):  
        next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)
        batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
        batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)

output = tokenizer.decode(batch['input_ids'][0, num_prompts:].cpu().numpy().tolist())
print(f"Output: {output}\n")

Output: A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it anyway! Besides, that



### Using HuggingFace PEFT (2 points)

[`peft`](https://huggingface.co/docs/peft/index) is a transformer's sister library that allows you to apply various __p__arameter __e__fficient __f__ine-__t__uning methods to pre-trained transformers. The library imlements both prompt tuning, prefix tuning, as well as several adapter-based techniques under a common interface:



In [20]:
!pip install -U bitsandbytes>=0.46.1

In [5]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 1.7 MB/s eta 0:00:0000:0100:01m


In [6]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",         
    bnb_4bit_use_double_quant=True,    
    bnb_4bit_compute_dtype=torch.bfloat16 
)

# del model
torch.cuda.empty_cache()
# gc.collect()
model_name = 'Enoch/llama-7b-hf'
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map='auto',
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
)

for param in model.parameters():
    param.requires_grad = False

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

import peft

peft_config = peft.PromptTuningConfig(
    task_type=peft.TaskType.CAUSAL_LM,
    num_virtual_tokens=32,  
    prompt_tuning_init=peft.PromptTuningInit.RANDOM,
)

model = peft.get_peft_model(model, peft_config)
print("Trainable parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))
print("Total parameters (excluding quantization):", sum(p.numel() for p in model.parameters()))

Loading weights: 100%|██████████| 291/291 [00:03<00:00, 84.11it/s, Materializing param=model.norm.weight]                               


Trainable parameters: 131072
Total parameters (excluding quantization): 3500544000


In [16]:
from peft import get_peft_model, PromptTuningConfig, TaskType

peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    num_virtual_tokens=32
)

model = get_peft_model(model, peft_config)

trainable_params = [p for p in model.parameters() if p.requires_grad]
opt = torch.optim.AdamW(trainable_params, lr=1e-2, fused=True)

num_params = sum(p.numel() for p in trainable_params)
print(f"Total trainable parameters: {num_params}")
if num_params == 0:
    raise ValueError("No trainable parameters found! Check your PEFT setup.")

model.train()
print(f"{'Step':<8} {'Loss':<12} {'Time (ms)'}")

for step in range(150):
    step_start = time.time()
    opt.zero_grad(set_to_none=True)
    
    with torch.amp.autocast('cuda', dtype=torch.bfloat16):
        outputs = model(**batch, labels=batch['input_ids'])
        loss = outputs.loss

    loss.backward()
    opt.step()
    
    torch.cuda.synchronize()
    step_time = (time.time() - step_start) * 1000
    
    if step % 5 == 0: 
        print(f"{step:<8} {loss.item():<12.4f} {step_time:.1f}")
    
    if loss.item() <= 0.05:
        print(f"\n Loss reached {loss.item():.4f} at step {step}")
        break

Total trainable parameters: 131072
Step     Loss         Time (ms)
0        7.0883       197.6
5        4.9757       141.5
10       3.3164       140.1
15       1.8947       141.2
20       0.8040       140.6
25       0.2368       137.4
30       0.0485       162.5

 Loss reached 0.0485 at step 30


In [17]:
prompt = 'A quick brown fox'
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)

with torch.no_grad():
    for i in range(20):
        next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)
        batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
        batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)

output = tokenizer.decode(batch['input_ids'][0].cpu().numpy().tolist())
print(f"Output: {output}\n")

Output: A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it anyway! Besides, that



##  LoRA: Low-Rank Adaptation (2 points)

LoRA adds trainable low-rank matrices in parallel with frozen weights.

<center><img src="https://i.imgur.com/6bQLNiG.png" width=240px></center>

In [5]:
import gc
gc.collect()
torch.cuda.empty_cache()
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  12884 MiB |  12897 MiB |  26185 MiB |  13301 MiB |
|       from large pool |  12884 MiB |  12884 MiB |  25734 MiB |  12850 MiB |
|       from small pool |      0 MiB |     13 MiB |    451 MiB |    450 MiB |
|---------------------------------------------------------------------------|
| Active memory         |  12884 MiB |  12897 MiB |  26185 MiB |  13301 MiB |
|       from large pool |  12884 MiB |  12884 MiB |  25734 MiB |

In [2]:
class LoRALayer(torch.nn.Module):
    def __init__(self, module, rank):
        super().__init__()
        self.module = module
        
        self.in_features = module.in_features if hasattr(module, 'in_features') else module.weight.shape[1]
        self.out_features = module.out_features if hasattr(module, 'out_features') else module.weight.shape[0]
        
        self.adapter_dtype = torch.bfloat16 
        device = next(module.parameters()).device
        
        self.adapter_A = torch.nn.Parameter(
            torch.randn(self.in_features, rank, device=device, dtype=self.adapter_dtype) * 0.02
        )
        self.adapter_B = torch.nn.Parameter(
            torch.zeros(rank, self.out_features, device=device, dtype=self.adapter_dtype)
        )

    def forward(self, x):
        return self.module(x) + (x.to(self.adapter_dtype) @ self.adapter_A) @ self.adapter_B

In [30]:
test_linear = nn.Linear(128, 128)
test_linear.weight.data[...] = torch.eye(128)
test_adapter = LoRALayer(test_linear, rank=8)

assert torch.allclose(test_adapter(torch.ones(1, 1, 128)), test_linear.bias + 1), "please check your forward pass"

test_adapter.adapter_A.data[...] = torch.linspace(0.1, -0.5, 128 * 8).view(128, 8)
test_adapter.adapter_B.data[...] = torch.linspace(0.5, -0.1, 128 * 8).view(8, 128)
test_linear.bias.data[...] = torch.linspace(1., -1., 128)

dummy_loss = F.mse_loss(test_adapter(torch.ones(1, 128) / 128).squeeze(), torch.linspace(-1, 1, 128))
assert torch.allclose(dummy_loss, torch.tensor(1.3711389), rtol=0, atol=1e-4), "Loss calculation failed"

dummy_loss.backward()
assert all(w.grad is not None for w in [test_adapter.adapter_A, test_adapter.adapter_B]), "some adapter weights have no grad"
assert torch.allclose(test_adapter.adapter_A.grad.sum(), torch.tensor(-0.60158), rtol=0, atol=1e-4), "Grad A wrong"
assert torch.allclose(test_adapter.adapter_B.grad.sum(), torch.tensor(0.9931), rtol=0, atol=1e-4), "Grad B wrong"

del dummy_loss, test_linear, test_adapter
torch.cuda.empty_cache()

print("All tests passed!")

All tests passed!


### Apply LoRA to the model

The code below applies LoRA adapters on top of Q/K/V linear layers in Llama attention. You may also choose to modify other layers:
* self_attn.o_proj - attention output projection
* mlp.up_proj, mlp.gate_proj, mlp.down_proj - transformer feedforward layers
* lm_head - output LM head

__Note:__ please scroll down for the homework task

In [7]:
lora_rank = 8

for i, layer in enumerate(model.model.layers):
    target_modules = {
        'q_proj': layer.self_attn.q_proj,
        'k_proj': layer.self_attn.k_proj,
        'v_proj': layer.self_attn.v_proj
    }
    
    for name, module in target_modules.items():
        if not hasattr(module, 'adapter_A'):
            wrapped_layer = LoRALayer(module, rank=lora_rank)
            
            if name == 'q_proj': layer.self_attn.q_proj = wrapped_layer
            elif name == 'k_proj': layer.self_attn.k_proj = wrapped_layer
            elif name == 'v_proj': layer.self_attn.v_proj = wrapped_layer

In [8]:
batch = tokenizer("Testing gradient flow", return_tensors='pt', return_token_type_ids=False).to(device)

with torch.cuda.amp.autocast(dtype=torch.bfloat16):
    out = model.forward(**batch)
    (out.logits.norm() / 100).backward()

for i, module in enumerate(model.modules()):
    if isinstance(module, LoRALayer):
        assert module.adapter_B.grad is not None, f"Gradient missing in layer {i}!"
        assert module.adapter_B.grad.norm().item() > 0, f"Zero gradient in layer {i}!"

model.zero_grad(set_to_none=True)

/tmp/ipykernel_6269/2385816259.py:3: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.bfloat16):


### (example) How to train your model

The example below shows how to train the LoRA adapters on a dummy dataset. You will need to run a _similar_ training task later.

__Note:__ please scroll down for the homework task

In [10]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 591.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 2.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 MB 34.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 42.2 MB/s eta 0:00:0000:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.9/193.9 kB 45.6 MB/s eta 0:00:00
 

In [13]:
import datasets
data = datasets.load_dataset("Abirate/english_quotes", split="train[:32]") 
data = data.map(lambda samples: tokenizer(samples['quote']), batched=True)
model._hf_peft_config_loaded = True  

trainer = transformers.Trainer(
    model=model, 
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2, 
        gradient_accumulation_steps=1,
        warmup_steps=100, 
        max_steps=100, 
        learning_rate=2e-4, 
        bf16=True,            
        report_to="none",     
        save_strategy="no",   
        logging_steps=1, 
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

model.config.use_cache = False 

trainer.train()

# NOTE: this is just an example! you do not have to wait for this progressbar to finish :)

Map: 100%|██████████| 32/32 [00:00<00:00, 2434.97 examples/s]


Step,Training Loss
1,0.600244
2,0.211341
3,0.747742
4,0.277206
5,0.360437
6,1.037606
7,1.171049
8,0.726156
9,0.335734
10,0.806300


TrainOutput(global_step=100, training_loss=0.6858807059749961, metrics={'train_runtime': 21.1666, 'train_samples_per_second': 9.449, 'train_steps_per_second': 4.724, 'total_flos': 626893241204736.0, 'train_loss': 0.6858807059749961, 'epoch': 6.25})

### Final task: *actually* train the model (4 points)

Your task is to fine-tune the model to _generate python code_. Please use the above examples for inspiration. More specifically,

* __dataset:__ use [codeparrot-clean](https://huggingface.co/datasets/codeparrot/codeparrot-clean) or any other data containing python code. Since you do not need much data for this excercise, it is enough to use just shorter validation subset of `codeparrots`
* __preprocessing:__ select python code based on file extentions (.py)  (may skip in case of codeparrot - it is 100% python)
* __short lines:__ please take the first 512 characters of each line
* __adapter type:__ please use LoRA as defined above __plus at least one of:__
   - extra adapter on lm_head
   - extra adapter on MLP components (mlp.*)
   - trainable input embeddings (requires tweaking memory usage)

* __training:__ you do not have to train to convergence. If all goes well, your model should `.generate` code after 500 steps. Please use batch size of at least 4 (4 x 1 x 512 tokens) using `gradient_accumulation_steps=4`.


Note: the peft library also has LoRA implementation. However, we ask that for this assignment you show at least one complete training run with your own LoRA code.

__Alternative assignment:__ Instead of doing python code, feel free to substitute the task with any other dataset, e.g. your favorite artist or podcast, as long as it's ethical. If you choose your own task, please show examples of what your model learned - or did not learn, akin to the code examples below.

In [36]:
model.eval()
model.gradient_checkpointing_disable() 
model.config.use_cache = True 

prompts = ['', 'import', 'from', 'while', 'try', 'if', 'for', 'torch', 'def', 'class']
baseline_outputs = {}

for prompt in prompts:
    if prompt == '':
        input_ids = torch.tensor([[tokenizer.bos_token_id]], device=model.device)
        attention_mask = torch.tensor([[1]], device=model.device)
        batch = {'input_ids': input_ids, 'attention_mask': attention_mask}
    else:
        batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(model.device)
    
    with torch.no_grad():
        with torch.amp.autocast('cuda', dtype=torch.bfloat16):
            for i in range(60):
                outputs = model(**batch)
                
                next_token_logits = outputs.logits[0, -1, :]
                next_token = next_token_logits.argmax(-1).reshape(1, 1)
                
                batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
                batch['attention_mask'] = torch.cat([
                    batch['attention_mask'], 
                    torch.ones_like(next_token)
                ], dim=-1)
    
    output_text = tokenizer.decode(batch['input_ids'][0], skip_special_tokens=True)
    baseline_outputs[prompt] = output_text
    
    display_prompt = prompt if prompt else "<empty string>"
    print(f"\nPrompt: '{display_prompt}'")
    print(f"Output: {output_text[:150]}...")



Prompt: '<empty string>'
Output: The 2018-2019 school year is off to a great start! We are excited to welcome our new students and families to the school. We are also excited to welco...

Prompt: 'import'
Output: import of the 1990s.
The 1990s were a time of great change in the world of fashion. The 1980s were a time of excess, and the 1990s were a time of mini...

Prompt: 'from'
Output: from the 1960s.
The 1960s were a time of great change in the United States. The Civil Rights Movement was in full swing, and the Vietnam War was ragin...

Prompt: 'while'
Output: while the other is a 100% cotton.
The 100% cotton is a bit more expensive, but it is also more durable.
The 100% cotton is also a bit more breathable,...

Prompt: 'try'
Output: try to get the best out of the players.
“I’m not going to be a manager who is going to be shouting and screaming at the players.
“I’m going to be a ma...

Prompt: 'if'
Output: if you are a fan of the genre.
The game is a 2D platformer, and it is a very

In [37]:
import datasets
import hashlib

code_data = datasets.load_dataset(
    "huggingface-course/codeparrot-ds-train", 
    split="train", 
    streaming=True
)

code_samples = []
seen_hashes = set()
target_samples = 1000 

print("Starting data collection...")

for entry in code_data:
    content = entry['content']
    
    content_hash = hashlib.md5(content.encode('utf-8')).hexdigest()
    
    if content_hash not in seen_hashes:
        seen_hashes.add(content_hash)
        code_samples.append(content)
        
        if len(code_samples) % 100 == 0:
            print(f"Collected {len(code_samples)} samples...")
            
    if len(code_samples) >= target_samples:
        break

Starting data collection...
Collected 100 samples...
Collected 200 samples...
Collected 300 samples...
Collected 400 samples...
Collected 500 samples...
Collected 600 samples...
Collected 700 samples...
Collected 800 samples...
Collected 900 samples...
Collected 1000 samples...


In [39]:
os.environ["PYTORCH_ALLOC_CONF"] = "expandable_segments:True"

def tokenize_function(examples):
    return tokenizer(
        examples["content"], 
        truncation=True, 
        max_length=512, 
        padding=False    
    )

tokenized_data = code_data.map(
    tokenize_function, 
    batched=True, 
    remove_columns=["content"] 
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_data, 
    args=transformers.TrainingArguments(
        max_steps=500,                   
        per_device_train_batch_size=2,   
        gradient_accumulation_steps=8,    
        learning_rate=2e-4,
        bf16=True,
        optim="adamw_torch_fused",
        gradient_checkpointing=True,
        dataloader_num_workers=0, 
        logging_steps=10,
        output_dir='outputs/python_code',
        save_strategy="no",              
        report_to="none"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
model.gradient_checkpointing_enable()
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.


Step,Training Loss
10,1.258569
20,1.268279
30,1.210268
40,1.253846
50,1.225961
60,1.246376
70,1.195075
80,1.225357
90,1.282946
100,1.276480


TrainOutput(global_step=500, training_loss=1.240020746231079, metrics={'train_runtime': 7568.8193, 'train_samples_per_second': 1.057, 'train_steps_per_second': 0.066, 'total_flos': 1.6232586142733107e+17, 'train_loss': 1.240020746231079, 'epoch': 1.0})

In [49]:
model.eval()
model.config.use_cache = True

finetuned_outputs = {}

for prompt in prompts:
    if not prompt:
        input_ids = torch.tensor([[tokenizer.bos_token_id or 1]], device=model.device)
        attention_mask = torch.tensor([[1]], device=model.device)
        batch = {'input_ids': input_ids, 'attention_mask': attention_mask}
    else:
        batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(model.device)
    
    with torch.no_grad():
        with torch.amp.autocast('cuda', dtype=torch.bfloat16):
            for i in range(60):
                outputs = model(**batch)
                
                next_token_logits = outputs.logits[0, -1, :]
                next_token = next_token_logits.argmax(-1).reshape(1, 1)
                
                batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
                batch['attention_mask'] = torch.cat([
                    batch['attention_mask'], 
                    torch.ones_like(next_token)
                ], dim=-1)
                
                del outputs
    
    output_text = tokenizer.decode(batch['input_ids'][0], skip_special_tokens=True)
    finetuned_outputs[prompt] = output_text
    
    print(f"\nPrompt: '{prompt if prompt else '<empty>'}'")
    print(f"Output: {output_text[:150]}...")
    
    del batch
    torch.cuda.empty_cache()


Prompt: '<empty>'
Output: The 2018-2019 school year is off to a great start! We are excited to welcome our new students and families to the school. We are also excited to welco...

Prompt: 'import'
Output: import of the 1990s.
The 1990s were a time of great change in the world of fashion. The 1980s were a time of excess, and the 1990s were a time of mini...

Prompt: 'from'
Output: from the 1960s.
The 1960s were a time of great change in the United States. The Civil Rights Movement was in full swing, and the Vietnam War was ragin...

Prompt: 'while'
Output: while the other is a 100% cotton.
The 100% cotton is a bit more expensive, but it is also more durable.
The 100% cotton is also a bit more breathable,...

Prompt: 'try'
Output: try to get the best out of the players.
“I’m not going to be a manager who is going to be shouting and screaming at the players.
“I’m going to be a ma...

Prompt: 'if'
Output: if you are a fan of the genre.
The game is a 2D platformer, and it is a very good o

In [7]:
import os
import gc
import torch
import hashlib
import datasets
import transformers
from transformers import LlamaTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
os.environ['LD_LIBRARY_PATH'] = '/opt/conda/lib/'
os.environ["PYTORCH_ALLOC_CONF"] = "expandable_segments:True"

model_name = 'Enoch/llama-7b-hf'

tokenizer = LlamaTokenizer.from_pretrained(model_name, legacy=True, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16 
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

class LoRALayer(torch.nn.Module):
    def __init__(self, module, rank):
        super().__init__()
        self.module = module
        self.in_features = module.in_features if hasattr(module, 'in_features') else module.weight.shape[1]
        self.out_features = module.out_features if hasattr(module, 'out_features') else module.weight.shape[0]
        
        self.dtype = torch.bfloat16 
        target_device = next(module.parameters()).device
        
        self.adapter_A = torch.nn.Parameter(
            torch.randn(self.in_features, rank, device=target_device, dtype=self.dtype) * 0.02
        )
        self.adapter_B = torch.nn.Parameter(
            torch.zeros(rank, self.out_features, device=target_device, dtype=self.dtype)
        )

    def forward(self, x):
        return self.module(x) + (x.to(self.dtype) @ self.adapter_A) @ self.adapter_B


for param in model.parameters():
    param.requires_grad = False

for layer in model.model.layers:
    if not hasattr(layer.self_attn.q_proj, 'adapter_A'):
        layer.self_attn.q_proj = LoRALayer(layer.self_attn.q_proj, rank=8)
        layer.self_attn.v_proj = LoRALayer(layer.self_attn.v_proj, rank=8)

for n, p in model.named_parameters():
    if "adapter_" in n:
        p.requires_grad = True

model.enable_input_require_grads()
model.gradient_checkpointing_enable()
model.is_quantized = False
model.peft_config = {"dummy": "dummy"} 

raw_data = datasets.load_dataset("huggingface-course/codeparrot-ds-train", split="train", streaming=True)

def tokenize_function(examples):
    return tokenizer(examples["content"], truncation=True, max_length=512, padding=False)

tokenized_data = raw_data.map(tokenize_function, batched=True, remove_columns=["content"])

trainable_params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(trainable_params, lr=2e-4, fused=True)

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_data, 
    args=transformers.TrainingArguments(
        max_steps=200,                   
        per_device_train_batch_size=2,   
        gradient_accumulation_steps=8,    
        learning_rate=2e-4,
        bf16=True,                       
        optim="adamw_torch_fused",
        dataloader_num_workers=0,        
        logging_steps=10,
        output_dir='outputs/python_code',
        save_strategy="no",              
        report_to="none"
    ),
    optimizers=(optimizer, None),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

model.eval()
model.gradient_checkpointing_disable()
model.config.use_cache = True

prompts = ['', 'import', 'from', 'while', 'try', 'if', 'for', 'torch', 'def', 'class']

print("\n" + "="*50)
print("GENERATION RESULTS")
print("="*50)

for prompt in prompts:
    if not prompt:
        bos_id = tokenizer.bos_token_id if tokenizer.bos_token_id is not None else 1
        input_ids = torch.tensor([[bos_id]], device=model.device)
        attention_mask = torch.tensor([[1]], device=model.device)
        batch = {'input_ids': input_ids, 'attention_mask': attention_mask}
    else:
        batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(model.device)
    
    with torch.no_grad():
        with torch.amp.autocast('cuda', dtype=torch.bfloat16):
            for i in range(60):
                outputs = model(**batch)
                next_token = outputs.logits[0, -1, :].argmax(-1).reshape(1, 1)
                batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
                batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)
                del outputs
    
    output_text = tokenizer.decode(batch['input_ids'][0], skip_special_tokens=True)
    
    clean_text = output_text[:120].replace('\n', ' ')
    disp_p = prompt if prompt else "<empty>"
    print(f"Prompt: '{disp_p}' -> {clean_text}...")
    
    del batch
    torch.cuda.empty_cache()
    gc.collect()

Loading weights: 100%|██████████| 291/291 [00:02<00:00, 110.69it/s, Materializing param=model.norm.weight]                              


Step,Training Loss
10,1.250538
20,1.207693
30,1.118477
40,1.156401
50,1.117085
60,1.142845
70,1.087480
80,1.111781
90,1.165279
100,1.162860



GENERATION RESULTS
Prompt: '<empty>' -> ﻿using System; using System.Collections.Generic; using System.Linq; using System.Text; using System.Threading.Tasks;  na...
Prompt: 'import' -> import numpy as np import matplotlib.pyplot as plt  from sklearn.datasets import load_iris from sklearn.ensemble import ...
Prompt: 'from' -> from __future__ import print_function  import numpy as np import matplotlib.pyplot as plt from matplotlib import cm from...
Prompt: 'while' -> while __pyx_t_1 = (((__pyx_v_info->flags & __pyx_v_info->flags_valid) != 0) != 0);             __pyx_t_2 = __pyx_t_1;   ...
Prompt: 'try' -> try sys.settrace()   def _get_trace_func():     """     Returns the trace function to use.     """     try:         impo...
Prompt: 'if' -> if __name__ == '__main__':      import sys     import os     import numpy as np     import matplotlib.pyplot as plt     ...
Prompt: 'for' -> for __pyx_t_1 = 0; __pyx_t_1 < __pyx_t_2; __pyx_t_1+=1) { #if CYTHON_COMPILING_IN_CPYTHON    #...
Prompt: 't

In [36]:
from IPython.display import display, HTML

rows = []
for i, prompt in enumerate(generation_results['prompts']):
    before_raw = generation_results['baseline'].get(prompt, "No data found")
    after_raw = generation_results['finetuned'].get(prompt, "No data found")
    
    before = before_raw.replace('<', '&lt;').replace('>', '&gt;')[:250]
    after = after_raw.replace('<', '&lt;').replace('>', '&gt;')[:250]
    
    bg = '#ecf0f1' if i % 2 == 0 else '#ffffff'
    
    row = f'''<tr style="background-color: {bg};">
        <td style="padding: 10px; border: 1px solid #bdc3c7; font-weight: bold; vertical-align: top;">'{prompt}'</td>
        <td style="padding: 10px; border: 1px solid #bdc3c7; white-space: pre-wrap; font-size: 11px; vertical-align: top;">{before}</td>
        <td style="padding: 10px; border: 1px solid #bdc3c7; white-space: pre-wrap; font-size: 11px; vertical-align: top; background-color: #e8f8f5;">{after}</td>
    </tr>'''
    rows.append(row)

table_html = f'''
<table style="width: 100%; border-collapse: collapse; font-family: sans-serif; border: 1px solid #bdc3c7;">
    <thead>
        <tr style="background-color: #2c3e50; color: #ffffff;">
            <th style="padding: 12px; text-align: left; width: 10%;">PROMPT</th>
            <th style="padding: 12px; text-align: left; width: 45%;">BEFORE TRAINING</th>
            <th style="padding: 12px; text-align: left; width: 45%;">AFTER TRAINING (H200)</th>
        </tr>
    </thead>
    <tbody>
        {"".join(rows)}
    </tbody>
</table>
'''

display(HTML(table_html))

PROMPT,BEFORE TRAINING,AFTER TRAINING (H200)
'',The 2018-2019 school year is off to a great start! We are excited to welcome our new students and families to the school. We are also excited to welco...,### Liens externes * (en) Référence Animal Diversity Web : Acanthoscurus (consulté le 19 septembre 2014) * (en) Référence Catalogue of Life : Acanthoscurus (consulté le 12
'import',"import of the 1990s. The 1990s were a time of great change in the world of fashion. The 1980s were a time of excess, and the 1990s were a time of mini...",import numpy as np import matplotlib.pyplot as plt from sklearn.datasets import load_iris from sklearn.ensemble import RandomForestClassifier from sklearn.metrics import confusion_matrix # Load the iris dataset iris = load
'from',"from the 1960s. The 1960s were a time of great change in the United States. The Civil Rights Movement was in full swing, and the Vietnam War was ragin...",from __future__ import division from __future__ import print_function from __future__ import unicode_literals import numpy as np import matplotlib.pyplot as plt from sklearn.datasets import load_iris from sklearn.ensemble
'while',"while the other is a 100% cotton. The 100% cotton is a bit more expensive, but it is also more durable. The 100% cotton is also a bit more breathable,...",while __pyx_t_1 = (((__pyx_v_info->flags & __pyx_v_info->flags_valid) != 0) != 0); __pyx_t_2 = __pyx_t_1;
'try',try to get the best out of the players. “I’m not going to be a manager who is going to be shouting and screaming at the players. “I’m going to be a ma...,"try sys.settrace() def _get_trace_func(): """""" Returns the trace function to use. """""" try: import ctypes return ctypes.CDLL('libpython.so.1.0"
'if',"if you are a fan of the genre. The game is a 2D platformer, and it is a very good one. The game is very well designed, and the controls are very respo...",if __name__ == '__main__': import sys import os import numpy as np import matplotlib.pyplot as plt import matplotlib.cm as cm import matplotlib.colors as colors import matplotlib.patches as patches
'for',for the first time in 2017. The 2017 edition of the event will be held from 10 to 12 November 2017 at the Dubai International Convention and Exhibitio...,for __pyx_t_1 = 0; __pyx_t_1 < __pyx_t_2; __pyx_t_1+=1) { #else #if CYTHON_COMPILING_IN_CPYTHON
'torch',"torch, and the torch is a symbol of the light of the Holy Spirit. The light of the Holy Spirit is the light of truth. The light of the Holy Spirit is ...","torch. The 1990s saw the rise of the Internet, and the 2000s saw the rise of the smartphone. The 2010s have seen the rise of the smartwatch. The smartwatch is a device that is worn on the"
'def',"defendant's motion to dismiss the indictment. The indictment charged that the defendant, on or about August 1, 1958, did unlawfully, wilfully and know...","def __init__(self, *args, **kwargs): super(TestCase, self).__init__(*args, **kwargs) self.assertTrue(self.assertTrue is self.assertTrue) self.assertFalse(self.assertFalse is self"
'class',classroom. The first step in the process is to create a classroom culture that is conducive to learning. This means that the teacher must be able to c...,class - the class that is the parent of the class that is the parent of the class that is the parent of the class that is the parent of the class that is the parent of the class that is the parent of the class that is the parent of the class that is


If you reach this: congratulations! you've completed everything in this practice session.

If you want to dig deeper, try to implement prompt-tuning (for bonus points!).
You can read more about prompt tuning variants in paper [1](https://arxiv.org/abs/2104.08691) or paper [2](https://arxiv.org/abs/2101.00190). Both versions can be implemented by passing trainable prompts as `model.forward(..., past_key_values=your_prompts)`.



### Read more

* How post-training quantization works: https://arxiv.org/abs/2208.07339
* An overview of running large models: https://huggingface.co/docs/accelerate/package_reference/big_modeling
* A general library for different adapter types: https://adapterhub.ml/


### [extra info] Running other models.

This notebook's code can run with other models of similar size, such as [Falcon-7B](https://huggingface.co/tiiuae/falcon-7b), [OPT-6.7B](https://huggingface.co/facebook/opt-6.7b) or [BLOOM-7.1B](https://huggingface.co/bigscience/bloom-7b1). However, they will require minor code tweaks:
1. change the model name in `AutoModelForCausalLM.from_pretrained()` __and__ `AutoTokenizer`
2. In the prompt tuning code, change `model.model.embed_tokens` to refer to the target model's word embeddings. Simply `print(model)` to navigate to them.
3. Change code to add Lora layers - specifically where you what the transformer block components, since those components now have different names.